# SWB Prospect Theory Modeling


Created: 06/20/2023 by Alie Fink

In [1]:
import numpy as np
import random
import pandas as pd 
import matplotlib.pyplot as plt
from scipy.optimize import minimize # minimize function is used for parameter recovery 
import seaborn as sns 
import tqdm
from scipy.stats import pearsonr
import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('/Users/alexandrafink/Documents/GraduateSchool/SaezLab/SWB/swb_computational_modeling/swb_behav_models/scripts/')
import SWB_modeling_utils 

## Find Optim Params

In [5]:
swb_dir = '/Users/alexandrafink/Documents/GraduateSchool/SaezLab/SWB/'
subj_list = pd.read_excel(f'{swb_dir}SWB_subjects.xlsx', sheet_name='Usable_Subjects', usecols=[0])
subj_ids = list(subj_list.PatientID)

In [6]:
subj_ids

['DA8',
 'DA9',
 'DA10',
 'DA11',
 'DA023',
 'MS002',
 'MS003',
 'MS016',
 'MS017',
 'MS019',
 'MS022',
 'MS025',
 'MS026',
 'MS027',
 'MS029',
 'MS030']

In [7]:
model_input_path = '/Users/alexandrafink/Documents/GraduateSchool/SaezLab/SWB/swb_computational_modeling/swb_behav_models/data/model_input_data_06192023'
model_input = pd.read_csv(model_input_path)

In [8]:
model_input

,subj_id,round,rate,zscore_rate,cr(t-1),cr(t-2),cr(t-3),choice_ev(t-1),choice_ev(t-2),choice_ev(t-3),...,totalregret(t-3),decisionregret(t-1),decisionregret(t-2),decisionregret(t-3),totalrelief(t-1),totalrelief(t-2),totalrelief(t-3),decisionrelief(t-1),decisionrelief(t-2),decisionrelief(t-3)
0,DA8,4,74.0,-1.156527,0.0,0.0,0.0,0.00,0.03,0.000,...,0.00,-1.50,0.00,0.00,0.00,0.66,0.80,0.0,0.36,0.80
1,DA8,7,80.0,0.054333,-0.3,-0.3,0.0,0.00,0.00,0.315,...,0.00,-0.30,0.00,0.00,0.00,0.30,0.63,0.0,0.30,0.43
2,DA8,10,84.0,0.861574,0.6,0.6,0.0,0.00,0.50,0.000,...,0.00,0.00,0.00,0.00,0.60,1.00,0.80,0.6,0.80,0.80
3,DA8,13,88.0,1.668814,-0.5,-0.5,0.0,0.00,1.50,0.000,...,-0.16,-0.50,0.00,-0.16,0.00,3.00,0.00,0.0,2.40,0.00
4,DA8,16,89.0,1.870624,0.0,0.0,-0.5,0.00,0.00,0.000,...,0.00,0.00,0.00,0.00,0.30,1.08,0.90,0.3,1.08,0.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,MS030,139,47.0,0.261100,0.0,0.0,0.0,0.56,-0.56,0.000,...,0.00,-0.40,-0.72,0.00,0.00,0.00,1.50,0.0,0.00,1.50
796,MS030,142,46.0,-0.372999,0.0,0.0,0.0,-0.79,0.00,0.000,...,0.00,-1.08,0.00,0.00,0.00,0.80,0.30,0.0,0.80,0.30
797,MS030,145,46.0,-0.372999,0.0,0.0,0.0,-0.42,0.00,0.000,...,-0.41,-0.34,0.00,-0.41,0.00,0.20,0.00,0.0,0.20,0.00
798,MS030,148,47.0,0.261100,0.0,0.0,0.0,-0.42,0.90,0.315,...,0.00,-0.54,0.00,0.00,0.00,1.80,0.63,0.0,1.30,0.43


### Model 1: Base PT Model
- free params: risk aversion, loss aversion, temp
- bounds = (0.1,3),(0.1,5),(0.1,20)
- beta distrubution assumptions = [1,9],[2,4],[1,8]

distribution assumptions estimated from Charpentier et al 2016 DATA_for_JASP_Anxiety.xlsx


In [36]:
risk_inits = SWB_modeling_utils.param_init(n_values=1, n_iter=5, upper_bound=3, lower_bound=0.1, method='beta',beta_shape=[1,9])
loss_inits = SWB_modeling_utils.param_init(n_values=1, n_iter=5, upper_bound=4, lower_bound=0.1, method='beta',beta_shape=[2,4])
temp_inits = SWB_modeling_utils.param_init(n_values=1, n_iter=5, upper_bound=20, lower_bound=0.1, method='beta',beta_shape=[1,8])

In [ ]:
swb_base_pt_dict = {}
bounds=(0.1,3),(0.1,4),(0.1,20)

for subj_id in subj_ids:
    df = pd.read_csv(f'{swb_dir}behavior_analysis/behavior_preprocessed/{subj_id}_task_data')
    risk_est, loss_est, temp_est, bic_est, optim_inits = SWB_modeling_utils.run_base_pt(df,risk_inits,loss_inits,temp_inits,bounds=bounds)
    if risk_est == 0:
        print(subj_id)
    swb_base_pt_dict[subj_id] = {'risk_estimation':risk_est,
                                'loss_estimation':loss_est,
                                'temp_estimation':temp_est,
                                'bic_estimation':bic_est,
                                'optim_inits':optim_inits}


In [39]:
risk_inits = SWB_modeling_utils.param_init(n_values=1, n_iter=5, upper_bound=3, lower_bound=0.1, method='beta',beta_shape=[1,9])
loss_inits = SWB_modeling_utils.param_init(n_values=1, n_iter=5, upper_bound=4, lower_bound=0.1, method='beta',beta_shape=[2,4])
temp_inits = SWB_modeling_utils.param_init(n_values=1, n_iter=5, upper_bound=20, lower_bound=0.1, method='beta',beta_shape=[1,8])
print(risk_inits,loss_inits,temp_inits)

[0.26361451 0.10668719 0.33909693 0.45696763 0.34206114] [1.45372477 0.75793349 0.57393343 0.37239053 1.79138866] [2.14422819 0.4040581  0.962921   0.50246531 0.188585  ]


In [ ]:
# subj_id = 'MS025'
# df = pd.read_csv(f'{swb_dir}behavior_analysis/behavior_preprocessed/{subj_id}_task_data')
# risk_est, loss_est, temp_est, bic_est, optim_inits = SWB_modeling_utils.run_base_pt(df,risk_inits,loss_inits,temp_inits,bounds=bounds)
# print(risk_est,loss_est,bic_est,temp_inits)

debug pt1

In [48]:
risk_inits = SWB_modeling_utils.param_init(n_values=1, n_iter=5, upper_bound=3, lower_bound=0.1, method='beta',beta_shape=[1,9])
loss_inits = SWB_modeling_utils.param_init(n_values=1, n_iter=5, upper_bound=4, lower_bound=0.1, method='beta',beta_shape=[2,4])
temp_inits = SWB_modeling_utils.param_init(n_values=1, n_iter=5, upper_bound=20, lower_bound=0.1, method='beta',beta_shape=[1,8])
print(risk_inits,loss_inits,temp_inits)

[0.12405224 0.20046404 0.47409242 0.11141767 0.20446688] [0.91409761 1.88177556 0.2253611  1.42627017 1.10411835] [0.20999594 1.74911005 2.30996617 3.22424843 0.56821895]


In [51]:
subj_id = 'DA9'
df = pd.read_csv(f'{swb_dir}behavior_analysis/behavior_preprocessed/{subj_id}_task_data')
df
#risk_est, loss_est, temp_est, bic_est, optim_inits = SWB_modeling_utils.run_base_pt(df,risk_inits,loss_inits,temp_inits,bounds=bounds)

,Round,Trial Num,TrialType,TrialOnset,ChoiceOnset,DecisionOnset,FeedbackOnset,RT,SafeBet,LowBet,...,GambleEV,CR,choiceEV,RPE,totalCPE,decisionCPE,totalRegret,decisionRegret,totalRelief,decisionRelief
0,1,45.0,mix,458.262575,458.273875,462.041225,462.049006,3.767350,0.0,-1.10,...,0.110,0.0,0.000,0.000,-1.32,-1.32,0.00,0.0,1.10,1.10
1,2,146.0,loss,468.814804,468.828686,473.819151,473.826753,4.990465,-0.6,-1.68,...,-0.840,-0.6,0.000,0.000,-0.60,-0.60,-0.60,-0.6,0.00,0.00
2,3,140.0,loss,480.509956,480.510710,484.830998,484.838941,4.320288,-0.5,-2.50,...,-1.250,-0.5,0.000,0.000,-0.50,-0.50,0.00,0.0,2.00,2.00
3,4,57.0,gain,497.135665,497.142355,500.407262,502.416627,3.264907,0.2,0.00,...,0.315,0.0,0.315,0.315,0.00,0.43,0.00,0.0,0.63,0.43
4,5,47.0,mix,507.397588,507.406724,511.335869,511.343683,3.929145,0.0,-1.10,...,0.550,0.0,0.000,0.000,-2.20,-2.20,0.00,0.0,1.10,1.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,146,127.0,loss,2211.392241,2211.392976,2214.280426,2214.288188,2.887450,-0.4,-1.26,...,-0.630,-0.4,0.000,0.000,-0.40,-0.40,-0.40,-0.4,0.00,0.00
146,147,78.0,gain,2220.904660,2220.905430,2225.208157,2227.218260,4.302726,0.4,0.00,...,0.720,0.0,0.720,-0.720,-1.44,-0.40,-1.44,-0.4,0.00,0.00
147,148,85.0,gain,2236.130865,2236.138261,2238.452797,2240.478945,2.314536,0.5,0.00,...,0.620,0.0,0.620,0.620,0.00,0.74,0.00,0.0,1.24,0.74
148,149,148.0,loss,2245.193557,2245.194353,2247.982401,2247.990169,2.788048,-0.6,-2.16,...,-1.080,-0.6,0.000,0.000,-0.60,-0.60,0.00,0.0,1.56,1.56


In [62]:

# guess several different starting points for rho
for risk_guess in risk_inits:
    print(risk_guess)
    for loss_guess in loss_inits:
        print(loss_guess)
        for temp_guess in temp_inits:
            print(temp_guess)
            init_guess = (risk_guess, loss_guess, temp_guess)

            print(init_guess)

0.12405223957536779
0.9140976145081163
0.20999593937756938
(0.12405223957536779, 0.9140976145081163, 0.20999593937756938)
1.7491100544100755
(0.12405223957536779, 0.9140976145081163, 1.7491100544100755)
2.309966165708887
(0.12405223957536779, 0.9140976145081163, 2.309966165708887)
3.2242484276535492
(0.12405223957536779, 0.9140976145081163, 3.2242484276535492)
0.5682189530337132
(0.12405223957536779, 0.9140976145081163, 0.5682189530337132)
1.881775558647382
0.20999593937756938
(0.12405223957536779, 1.881775558647382, 0.20999593937756938)
1.7491100544100755
(0.12405223957536779, 1.881775558647382, 1.7491100544100755)
2.309966165708887
(0.12405223957536779, 1.881775558647382, 2.309966165708887)
3.2242484276535492
(0.12405223957536779, 1.881775558647382, 3.2242484276535492)
0.5682189530337132
(0.12405223957536779, 1.881775558647382, 0.5682189530337132)
0.22536109947002
0.20999593937756938
(0.12405223957536779, 0.22536109947002, 0.20999593937756938)
1.7491100544100755
(0.12405223957536779,

In [63]:
print(risk_inits,loss_inits,temp_inits)
print(risk_guess,loss_guess,temp_guess)

[0.12405224 0.20046404 0.47409242 0.11141767 0.20446688] [0.91409761 1.88177556 0.2253611  1.42627017 1.10411835] [0.20999594 1.74911005 2.30996617 3.22424843 0.56821895]
0.2044668829652807 1.1041183494156956 0.5682189530337132


In [64]:
params = init_guess
risk_aversion, loss_aversion, inverse_temp = params
print(params)
print(risk_aversion, loss_aversion, inverse_temp)
print(risk_aversion)
print(loss_aversion)
print(inverse_temp)

(0.2044668829652807, 1.1041183494156956, 0.5682189530337132)
0.2044668829652807 1.1041183494156956 0.5682189530337132
0.2044668829652807
1.1041183494156956
0.5682189530337132


In [66]:
subj_df = (df)

res_nll = np.inf
res_nll

inf

In [89]:
bounds=(0.1,3),(0.1,4),(0.1,20)
choiceprob_list = []


In [67]:
risk_aversion, loss_aversion, inverse_temp = params

In [69]:
range(len(subj_df))
trial=0

In [70]:
    trial_info = subj_df.iloc[trial]
    high_bet = trial_info['HighBet']
    low_bet = trial_info['LowBet']
    safe_bet = trial_info['SafeBet']
    trial_type = trial_info['TrialType']
    choice = trial_info['GambleChoice']
    print(trial_info)

Round                      1
Trial Num               45.0
TrialType                mix
TrialOnset        458.262575
ChoiceOnset       458.273875
DecisionOnset     462.041225
FeedbackOnset     462.049006
RT                   3.76735
SafeBet                  0.0
LowBet                  -1.1
HighBet                 1.32
HighBetPos               top
GamblePos               left
ChoicePos              right
GambleChoice            safe
Outcome                 good
Profit                   0.0
TotalProfit             10.0
GambleEV                0.11
CR                       0.0
choiceEV                 0.0
RPE                      0.0
totalCPE               -1.32
decisionCPE            -1.32
totalRegret              0.0
decisionRegret           0.0
totalRelief              1.1
decisionRelief           1.1
Name: 0, dtype: object


In [71]:
print(high_bet)
print(low_bet)
print(safe_bet)
print(trial_type)
print(choice)

1.32
-1.1
0.0
mix
safe


In [72]:
# transform to high bet value to utility (gamble)
if high_bet > 0: #mix or gain trials
    weighted_high_bet = 0.5 * ((high_bet)**risk_aversion)
else: #loss trials
    weighted_high_bet = 0 

# transform to low bet value to utility (gamble)
if low_bet < 0: #loss and mix trials
    weighted_low_bet = -0.5 * loss_aversion * ((-low_bet)**risk_aversion)
    
else: #gain trials
    weighted_low_bet = 0 

util_gamble = weighted_high_bet + weighted_low_bet


In [74]:
high_bet

1.32

In [75]:
risk_aversion

0.2044668829652807

In [73]:
weighted_high_bet

0.5292043193352624

In [76]:
weighted_low_bet

-0.5629230928825413

In [78]:
util_gamble

-0.033718773547278924

In [80]:
# transform safe bet value to utility (safe)
if safe_bet >= 0: #gain or mix trials
    util_safe = (safe_bet)**risk_aversion
else: #loss trials
    util_safe = -loss_aversion * ((-safe_bet)**risk_aversion)


In [81]:
print(util_safe)

0.0


In [82]:
# convert EV to choice probabilities via softmax
p_gamble = np.exp(inverse_temp*util_gamble) / ( np.exp(inverse_temp*util_gamble) + np.exp(inverse_temp*util_safe) )
p_safe = np.exp(inverse_temp*util_safe) / ( np.exp(inverse_temp*util_gamble) + np.exp(inverse_temp*util_safe) )

In [83]:
print(p_gamble,p_safe)

0.49521023497216865 0.5047897650278315


In [84]:
choice

'safe'

In [91]:
# append probability of chosen options
if choice == 'gamble':
    choiceprob_list.append(p_gamble)
elif choice == 'safe':
    choiceprob_list.append(p_safe)
else:
    choiceprob_list.append(0)

In [92]:
choiceprob_list

[0.5047897650278315]

In [93]:
negLL = -np.sum(np.log(choiceprob_list))
print(negLL)

0.6836132432621236


In [94]:
negLL < res_nll

True

In [ ]:
res_nll = negLL
param_fits = 
risk_aversion, loss_aversion, inverse_temp = param_fits
optim_vars = init_guess

In [100]:
bounds

((0.1, 3), (0.1, 4), (0.1, 20))

In [109]:
# params = init_guess
# risk_aversion, loss_aversion, inverse_temp = params
# print(params)
init_list = [risk_aversion,loss_aversion,inverse_temp]
init_np = np.array([risk_aversion,loss_aversion,inverse_temp])
print(type(params))
print(type(init_guess))
print(type(init_list))
print(type(init_np))
print(init_np)

<class 'tuple'>
<class 'tuple'>
<class 'list'>
<class 'numpy.ndarray'>
[0.20446688 1.10411835 0.56821895]


In [101]:

result = minimize(negll_base_pt, 
                x0=init_guess, 
                args=subj_df, 
                method='L-BFGS-B',
                bounds=bounds) #should match bounds given to param_init 

# if current negLL is smaller than the last negLL,
# # then store current data
# if result.fun < res_nll:
#     res_nll = result.fun
#     param_fits = result.x
#     risk_aversion, loss_aversion, inverse_temp = param_fits
#     optim_vars = init_guess

In [110]:
result_list = minimize(negll_base_pt, 
                x0=init_list, 
                args=subj_df, 
                method='L-BFGS-B',
                bounds=bounds) #should match bounds given to param_init 

In [115]:
result_list.x

array([0.20446688, 1.10411835, 0.56821895])

In [116]:
result_np = minimize(negll_base_pt, 
                x0=init_np, 
                args=subj_df)

In [119]:
result_np.x

array([0.20446688, 1.10411835, 0.56821895])

In [103]:
init_guess

(0.2044668829652807, 1.1041183494156956, 0.5682189530337132)

In [104]:
result.x

array([0.20446688, 1.10411835, 0.56821895])

In [105]:
result.fun

inf

In [ ]:
if result.fun < res_nll:
    res_nll = result.fun
    param_fits = result.x
    risk_aversion, loss_aversion, inverse_temp = param_fits
    optim_vars = init_guess

In [120]:
negll_base_pt(params,subj_df)

inf

In [121]:
risk_aversion, loss_aversion, inverse_temp = params

In [123]:
# init list of choice prob predictions
choiceprob_list = []

#loop through trials
for trial in range(len(subj_df)):

    # get relevant trial info
    trial_info = subj_df.iloc[trial]
    high_bet = trial_info['HighBet']
    low_bet = trial_info['LowBet']
    safe_bet = trial_info['SafeBet']
    trial_type = trial_info['TrialType']
    choice = trial_info['GambleChoice']

    
    #for simulation df - should change simulation code to match behavior data
    # trial_info = subj_df.iloc[trial]
    # high_bet = trial_info['high_bet']
    # low_bet = trial_info['low_bet']
    # safe_bet = trial_info['safe_bet']
    # trial_type = trial_info['type']
    # choice = trial_info['choice_pred']

    # transform to high bet value to utility (gamble)
    if high_bet > 0: #mix or gain trials
        weighted_high_bet = 0.5 * ((high_bet)**risk_aversion)
    else: #loss trials
        weighted_high_bet = 0 
    
    # transform to low bet value to utility (gamble)
    if low_bet < 0: #loss and mix trials
        weighted_low_bet = -0.5 * loss_aversion * ((-low_bet)**risk_aversion)
        
    else: #gain trials
        weighted_low_bet = 0 
    
    util_gamble = weighted_high_bet + weighted_low_bet


    # transform safe bet value to utility (safe)
    if safe_bet >= 0: #gain or mix trials
        util_safe = (safe_bet)**risk_aversion
    else: #loss trials
        util_safe = -loss_aversion * ((-safe_bet)**risk_aversion)



    # convert EV to choice probabilities via softmax
    p_gamble = np.exp(inverse_temp*util_gamble) / ( np.exp(inverse_temp*util_gamble) + np.exp(inverse_temp*util_safe) )
    p_safe = np.exp(inverse_temp*util_safe) / ( np.exp(inverse_temp*util_gamble) + np.exp(inverse_temp*util_safe) )

    # if np.isnan(p_gamble): #when utility is too large, probabilities cannot be estimated 
    #     p_gamble = 0.99
    #     p_safe = 0.01
    # if np.isnan(p_safe):
    #     p_safe = 0.99
    #     p_gamble = 0.01
    

    # append probability of chosen options
    if choice == 'gamble':
        choiceprob_list.append(p_gamble)
    elif choice == 'safe':
        choiceprob_list.append(p_safe)
    else:
        choiceprob_list.append(0)

# compute the neg LL of choice probabilities across the entire task
negLL = -np.sum(np.log(choiceprob_list))

In [96]:

def negll_base_pt(params, subj_df):
    risk_aversion, loss_aversion, inverse_temp = params
    #if using log transform then 

    # init list of choice prob predictions
    choiceprob_list = []

    #loop through trials
    for trial in range(len(subj_df)):

        # get relevant trial info
        trial_info = subj_df.iloc[trial]
        high_bet = trial_info['HighBet']
        low_bet = trial_info['LowBet']
        safe_bet = trial_info['SafeBet']
        trial_type = trial_info['TrialType']
        choice = trial_info['GambleChoice']

        
        #for simulation df - should change simulation code to match behavior data
        # trial_info = subj_df.iloc[trial]
        # high_bet = trial_info['high_bet']
        # low_bet = trial_info['low_bet']
        # safe_bet = trial_info['safe_bet']
        # trial_type = trial_info['type']
        # choice = trial_info['choice_pred']

        # transform to high bet value to utility (gamble)
        if high_bet > 0: #mix or gain trials
            weighted_high_bet = 0.5 * ((high_bet)**risk_aversion)
        else: #loss trials
            weighted_high_bet = 0 
        
        # transform to low bet value to utility (gamble)
        if low_bet < 0: #loss and mix trials
            weighted_low_bet = -0.5 * loss_aversion * ((-low_bet)**risk_aversion)
            
        else: #gain trials
            weighted_low_bet = 0 
        
        util_gamble = weighted_high_bet + weighted_low_bet
    

        # transform safe bet value to utility (safe)
        if safe_bet >= 0: #gain or mix trials
            util_safe = (safe_bet)**risk_aversion
        else: #loss trials
            util_safe = -loss_aversion * ((-safe_bet)**risk_aversion)



        # convert EV to choice probabilities via softmax
        p_gamble = np.exp(inverse_temp*util_gamble) / ( np.exp(inverse_temp*util_gamble) + np.exp(inverse_temp*util_safe) )
        p_safe = np.exp(inverse_temp*util_safe) / ( np.exp(inverse_temp*util_gamble) + np.exp(inverse_temp*util_safe) )

        # if np.isnan(p_gamble): #when utility is too large, probabilities cannot be estimated 
        #     p_gamble = 0.99
        #     p_safe = 0.01
        # if np.isnan(p_safe):
        #     p_safe = 0.99
        #     p_gamble = 0.01
        

        # append probability of chosen options
        if choice == 'gamble':
            choiceprob_list.append(p_gamble)
        elif choice == 'safe':
            choiceprob_list.append(p_safe)
        else:
            choiceprob_list.append(0)

    # compute the neg LL of choice probabilities across the entire task
    negLL = -np.sum(np.log(choiceprob_list))
    
    if np.isnan(negLL):
        return np.inf
    else:
        return negLL, 
    


In [125]:
print(choiceprob_list)

[0.5047897650278315, 0.4461189184123311, 0.45855734439463186, 0.4624743890931515, 0.49651335779128386, 0.4955121585106131, 0.521205505484407, 0.4459708094926683, 0.45093300179202184, 0.5517883503075384, 0.5124763314230155, 0.5396022220547553, 0.46265137160254455, 0.48980203179099924, 0.5424145296264146, 0.5414156621880039, 0.4612684962772367, 0.5233279127969109, 0.4986356446894883, 0.5432201977787249, 0.4634729107747212, 0.4858710696659018, 0.5248527615158303, 0.49901822541067964, 0.44839738829062575, 0.45020081524955663, 0.48663814077875134, 0.5236854021407569, 0.49218720659373816, 0.4593198704496165, 0.4781407096921947, 0.5469349406722395, 0.5199039036949407, 0.45699089165069706, 0.4935823136649471, 0.5487244265042407, 0.5029675307325341, 0.4888988487841418, 0.4529420378981364, 0.45037631177082066, 0.4596854979756086, 0.4847363626468501, 0.49245978879205865, 0.4919662103490439, 0.5096815644786981, 0.46636987901586474, 0.43761931160137096, 0.45230584677342545, 0.5104507612714336, 0.49

In [ ]:

#log transform for input to optim - then untransform in negll eq 

# guesses for alpha, theta will change on each loop

# minimize neg LL
# result = minimize(negll_base_pt, 
#                 x0=init_guess, 
#                 args=subj_df, 
#                 method='L-BFGS-B',
#                 bounds=bounds) #should match bounds given to param_init 

# if current negLL is smaller than the last negLL,
# then store current data
if result.fun < res_nll:
    res_nll = result.fun
    param_fits = result.x
    risk_aversion, loss_aversion, inverse_temp = param_fits
    optim_vars = init_guess

In [ ]:

#loop through trials
for trial in range(len(subj_df)):

    # get relevant trial info
    trial_info = subj_df.iloc[trial]
    high_bet = trial_info['HighBet']
    low_bet = trial_info['LowBet']
    safe_bet = trial_info['SafeBet']
    trial_type = trial_info['TrialType']
    choice = trial_info['GambleChoice']

    
    #for simulation df - should change simulation code to match behavior data
    # trial_info = subj_df.iloc[trial]
    # high_bet = trial_info['high_bet']
    # low_bet = trial_info['low_bet']
    # safe_bet = trial_info['safe_bet']
    # trial_type = trial_info['type']
    # choice = trial_info['choice_pred']

    # transform to high bet value to utility (gamble)
    if high_bet > 0: #mix or gain trials
        weighted_high_bet = 0.5 * ((high_bet)**risk_aversion)
    else: #loss trials
        weighted_high_bet = 0 
    
    # transform to low bet value to utility (gamble)
    if low_bet < 0: #loss and mix trials
        weighted_low_bet = -0.5 * loss_aversion * ((-low_bet)**risk_aversion)
        
    else: #gain trials
        weighted_low_bet = 0 
    
    util_gamble = weighted_high_bet + weighted_low_bet


    # transform safe bet value to utility (safe)
    if safe_bet >= 0: #gain or mix trials
        util_safe = (safe_bet)**risk_aversion
    else: #loss trials
        util_safe = -loss_aversion * ((-safe_bet)**risk_aversion)



    # convert EV to choice probabilities via softmax
    p_gamble = np.exp(inverse_temp*util_gamble) / ( np.exp(inverse_temp*util_gamble) + np.exp(inverse_temp*util_safe) )
    p_safe = np.exp(inverse_temp*util_safe) / ( np.exp(inverse_temp*util_gamble) + np.exp(inverse_temp*util_safe) )

    # if np.isnan(p_gamble): #when utility is too large, probabilities cannot be estimated 
    #     p_gamble = 0.99
    #     p_safe = 0.01
    # if np.isnan(p_safe):
    #     p_safe = 0.99
    #     p_gamble = 0.01
    

    # append probability of chosen options
    if choice == 'gamble':
        choiceprob_list.append(p_gamble)
    elif choice == 'safe':
        choiceprob_list.append(p_safe)
    else:
        choiceprob_list.append(0)

# compute the neg LL of choice probabilities across the entire task
negLL = -np.sum(np.log(choiceprob_list))
print(negLL)
    

In [ ]:



                

if res_nll == np.inf:
    print('No solution for this patient')
    risk_aversion=0
    loss_aversion=0
    inverse_temp=0
    BIC=0
    optim_vars=0
    #return risk_aversion, loss_aversion, inverse_temp, BIC, optim_vars
else:
    BIC = len(init_guess) * np.log(len(subj_df)) + 2*res_nll
print(risk_aversion,loss_aversion,inverse_temp,BIC,optim_vars)



In [ ]:
risk_aversion, loss_aversion, inverse_temp = params
#if using log transform then 

# init list of choice prob predictions
choiceprob_list = []

#loop through trials
for trial in range(len(subj_df)):

    # get relevant trial info
    trial_info = subj_df.iloc[trial]
    high_bet = trial_info['HighBet']
    low_bet = trial_info['LowBet']
    safe_bet = trial_info['SafeBet']
    trial_type = trial_info['TrialType']
    choice = trial_info['GambleChoice']

    
    #for simulation df - should change simulation code to match behavior data
    # trial_info = subj_df.iloc[trial]
    # high_bet = trial_info['high_bet']
    # low_bet = trial_info['low_bet']
    # safe_bet = trial_info['safe_bet']
    # trial_type = trial_info['type']
    # choice = trial_info['choice_pred']

    # transform to high bet value to utility (gamble)
    if high_bet > 0: #mix or gain trials
        weighted_high_bet = 0.5 * ((high_bet)**risk_aversion)
    else: #loss trials
        weighted_high_bet = 0 
    
    # transform to low bet value to utility (gamble)
    if low_bet < 0: #loss and mix trials
        weighted_low_bet = -0.5 * loss_aversion * ((-low_bet)**risk_aversion)
        
    else: #gain trials
        weighted_low_bet = 0 
    
    util_gamble = weighted_high_bet + weighted_low_bet


    # transform safe bet value to utility (safe)
    if safe_bet >= 0: #gain or mix trials
        util_safe = (safe_bet)**risk_aversion
    else: #loss trials
        util_safe = -loss_aversion * ((-safe_bet)**risk_aversion)



    # convert EV to choice probabilities via softmax
    p_gamble = np.exp(inverse_temp*util_gamble) / ( np.exp(inverse_temp*util_gamble) + np.exp(inverse_temp*util_safe) )
    p_safe = np.exp(inverse_temp*util_safe) / ( np.exp(inverse_temp*util_gamble) + np.exp(inverse_temp*util_safe) )

    if np.isnan(p_gamble): #when utility is too large, probabilities cannot be estimated 
        p_gamble = 0.99
        p_safe = 0.01
    if np.isnan(p_safe):
        p_safe = 0.99
        p_gamble = 0.01
    

    # append probability of chosen options
    if choice == 'gamble':
        choiceprob_list.append(p_gamble)
    elif choice == 'safe':
        choiceprob_list.append(p_safe)
    else:
        choiceprob_list.append(0)

# compute the neg LL of choice probabilities across the entire task
negLL = -np.sum(np.log(choiceprob_list))
print(negLL)
    

## Run pt models with optim params

In [38]:
subjects = pd.read_excel('/Users/alexandrafink/Documents/GraduateSchool/SaezLab/SWB/SWB_subjects.xlsx', sheet_name=0,usecols='A')
behav_path = '/Users/alexandrafink/Documents/GraduateSchool/SaezLab/SWB/behavior_analysis/behavior_preprocessed/'

## Update model data for GLM inputs

In [33]:
model_input_path = '/Users/alexandrafink/Documents/GraduateSchool/SaezLab/SWB/swb_computational_modeling/swb_behav_models/data/model_input_data_06192023'
model_input = pd.read_csv(model_input_path)